In [ ]:
import json
import pandas as pd

# 1. Tải dữ liệu từ file JSON
with open("../llms4subjects/shared-task-datasets/GND/dataset/GND-Subjects-tib-core.json", "r", encoding="utf-8") as file:
    data = json.load(file)

# 2. Chuyển dữ liệu sang DataFrame
df = pd.json_normalize(data)

# 3. Chọn các trường quan trọng
fields_to_keep = [
    "Code", "Name", "Classification Name", "Alternate Name", 
    "Related Subjects", "Definition", "Source URL"
]
df = df[fields_to_keep]

# 4. Kiểm tra dữ liệu
print("Số lượng dòng dữ liệu:", len(df))
print("Dữ liệu mẫu:")
print(df.head())

# 5. Xử lý các cột chứa danh sách (nếu cần)
# Chuyển Alternate Name và Related Subjects thành dạng chuỗi, dễ xử lý hơn
df["Alternate Name"] = df["Alternate Name"].apply(lambda x: ", ".join(x) if isinstance(x, list) else "")
df["Related Subjects"] = df["Related Subjects"].apply(lambda x: ", ".join(x) if isinstance(x, list) else "")

# 6. Lưu dữ liệu đã xử lý ra file CSV để kiểm tra
df.to_csv("processed_gnd_data.csv", index=False, encoding="utf-8")
del(df)


In [ ]:
from torch.utils.data import DataLoader
from transformers import AdamW

# Tạo DataLoader từ tập dữ liệu
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16)

# Chuẩn bị optimizer
optimizer = AdamW(model.parameters(), lr=2e-5)

# Vòng lặp huấn luyện
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

for epoch in range(3):  # Số lượng epoch
    model.train()
    total_loss = 0
    for batch in train_loader:
        optimizer.zero_grad()
        inputs = {k: v.to(device) for k, v in batch.items() if k != "labels"}
        labels = batch["labels"].to(device)
        outputs = model(**inputs, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    print(f"Epoch {epoch + 1} - Loss: {total_loss / len(train_loader)}")

# Đánh giá mô hình (tùy chọn)
model.eval()
# ...
